In [1]:
import pandas as pd

In [2]:
import requests, re
from bs4 import BeautifulSoup

In [3]:
#remove comments because it confuses parser
comm = re.compile("<!--|-->")
def make_soup(url):
    page = requests.get(url).text
    soup = BeautifulSoup(comm.sub('', page), 'html.parser')
    return soup

In [4]:
COLUMN_NAMES =["PLAYER", "COLLEGE", "CONF", "GAMES_PLAYED", "GAMES_STARTED", "RECORD", "CMP", "ATT", "YDS", "TD", "INT", "RUSH_ATT", "RUSH_YDS", "RUSH_TD", "COLL_CMP", "COLL_ATT", "COLL_YDS", "COLL_TD", "COLL_INT", "COLL_RUSH_ATT", "COLL_RUSH_YDS", "COLL_RUSH_TDS"]

In [5]:
def get_college_data(url):
    data = {}
    soup = make_soup(url)
#     meta = soup.find(id='meta')
#     weight_div = meta.find('span', itemprop='weight')
#     if weight_div is not None:
#         data['WT'] = weight_div.get_text()
#     height_div = meta.find('span', itemprop='height')
#     if height_div is not None:
#         data['HT'] = height_div.get_text()

    passing = soup.find('table', id='passing')
    last_row = passing.tbody.find_all('tr')[-1]
    data['CONF'] = last_row.find_all('td')[1].get_text()
    tds = passing.tfoot.find_all('td')
    data['COLL_CMP'] = tds[5].get_text()
    data['COLL_ATT'] = tds[6].get_text()
    data['COLL_YDS'] = tds[8].get_text()
    data['COLL_TD'] = tds[11].get_text()
    data['COLL_INT'] = tds[12].get_text()

    rushing = soup.find('table', id='rushing').tfoot
    tds = rushing.find_all('td')
    data['COLL_RUSH_ATT'] = tds[5].get_text()
    data['COLL_RUSH_YDS'] = tds[6].get_text()
    data['COLL_RUSH_TDS'] = tds[8].get_text()

    return data

In [6]:
DRAFT_URL = 'https://www.pro-football-reference.com/play-index/draft-finder.cgi?request=1&year_min=1997&year_max=2017&draft_slot_min=1&draft_slot_max=500&pick_type=overall&pos%5B%5D=qb&conference=any&show=all&order_by=default'
page = requests.get(DRAFT_URL).text
soup = BeautifulSoup(page, 'html.parser')

In [7]:
table = soup.find(id='results')

In [8]:
data = []
for tr in table.tbody.find_all('tr', class_=lambda x: x != 'thead'):
    player = {}
    for i, td in enumerate(tr.find_all('td')):
        if i == 3:
            player["PLAYER"] = td.get_text();
        if i == 13:
            player["GAMES_PLAYED"] = td.get_text();
        if i == 14:
            player["GAMES_STARTED"] = td.get_text();
        if i == 15:
            player["RECORD"] = td.get_text();
        if i == 16:
            player["CMP"] = td.get_text();
        if i == 17:
            player["ATT"] = td.get_text();
        if i == 18:
            player["YDS"] = td.get_text();
        if i == 19:
            player["TD"] = td.get_text();
        if i == 20:
            player["INT"] = td.get_text();
        if i == 21:
            player["RUSH_ATT"] = td.get_text();
        if i == 22:
            player["RUSH_YDS"] = td.get_text();
        if i == 23:
            player["RUSH_TD"] = td.get_text();
        if i == 24:
            player["COLLEGE"] = td.get_text();
        if i == 25 and td.a is not None:
            print('gathering data for: ' + player['PLAYER'])
            player.update(get_college_data(td.a['href']))
            data.append(player)

df = pd.DataFrame(data, columns=COLUMN_NAMES)
df

gathering data for: Mitchell Trubisky
gathering data for: Patrick Mahomes
gathering data for: Deshaun Watson
gathering data for: DeShone Kizer
gathering data for: Davis Webb
gathering data for: C.J. Beathard
gathering data for: Joshua Dobbs
gathering data for: Nathan Peterman
gathering data for: Brad Kaaya
gathering data for: Chad Kelly
gathering data for: Jared Goff
gathering data for: Carson Wentz
gathering data for: Paxton Lynch
gathering data for: Christian Hackenberg
gathering data for: Jacoby Brissett
gathering data for: Cody Kessler
gathering data for: Connor Cook
gathering data for: Dak Prescott
gathering data for: Cardale Jones
gathering data for: Kevin Hogan
gathering data for: Nate Sudfeld
gathering data for: Jake Rudock
gathering data for: Brandon Allen
gathering data for: Jeff Driskel
gathering data for: Brandon Doughty
gathering data for: Jameis Winston
gathering data for: Marcus Mariota
gathering data for: Garrett Grayson
gathering data for: Sean Mannion
gathering data f

,PLAYER,COLLEGE,CONF,GAMES_PLAYED,GAMES_STARTED,RECORD,CMP,ATT,YDS,TD,...,RUSH_YDS,RUSH_TD,COLL_CMP,COLL_ATT,COLL_YDS,COLL_TD,COLL_INT,COLL_RUSH_ATT,COLL_RUSH_YDS,COLL_RUSH_TDS
0,Mitchell Trubisky,North Carolina,ACC,22,22,11-11-0,406,651,4662,27,...,611,5,386,572,4762,41,10,120,439,8
1,Patrick Mahomes,Texas Tech,Big 12,13,13,11-2-0,309,464,4207,41,...,248,2,857,1349,11252,93,29,308,845,22
2,Deshaun Watson,Clemson,ACC,19,18,12-6-0,368,568,4730,40,...,644,4,814,1207,10168,90,32,435,1934,26
3,DeShone Kizer,Notre Dame,Ind,16,15,0-15-0,259,483,2949,11,...,419,5,422,695,5805,47,19,264,997,18
4,Davis Webb,California,Pac-12,,,,,,,,...,,,841,1367,9852,83,34,77,-100,9
5,C.J. Beathard,Iowa,Big Ten,13,10,1-9-0,225,393,2682,12,...,205,4,454,782,5562,40,19,224,429,10
6,Joshua Dobbs,Tennessee,SEC,4,0,,2,3,19,0,...,-4,0,614,999,7138,53,29,438,2160,32
7,Nathan Peterman,Pittsburgh,ACC,8,4,1-3-0,68,130,548,3,...,73,1,398,662,5236,47,17,172,525,5
8,Brad Kaaya,Miami (FL),ACC,,,,,,,,...,,,720,1188,9968,69,24,94,-386,4
9,Chad Kelly,Mississippi,SEC,1,0,,,,,,...,-1,0,513,803,6858,50,21,203,958,16


In [9]:
df = df.replace('', 0)
df['RECORD'] = df['RECORD'].replace(0, '0-0-0')

In [10]:
df

,PLAYER,COLLEGE,CONF,GAMES_PLAYED,GAMES_STARTED,RECORD,CMP,ATT,YDS,TD,...,RUSH_YDS,RUSH_TD,COLL_CMP,COLL_ATT,COLL_YDS,COLL_TD,COLL_INT,COLL_RUSH_ATT,COLL_RUSH_YDS,COLL_RUSH_TDS
0,Mitchell Trubisky,North Carolina,ACC,22,22,11-11-0,406,651,4662,27,...,611,5,386,572,4762,41,10,120,439,8
1,Patrick Mahomes,Texas Tech,Big 12,13,13,11-2-0,309,464,4207,41,...,248,2,857,1349,11252,93,29,308,845,22
2,Deshaun Watson,Clemson,ACC,19,18,12-6-0,368,568,4730,40,...,644,4,814,1207,10168,90,32,435,1934,26
3,DeShone Kizer,Notre Dame,Ind,16,15,0-15-0,259,483,2949,11,...,419,5,422,695,5805,47,19,264,997,18
4,Davis Webb,California,Pac-12,0,0,0-0-0,0,0,0,0,...,0,0,841,1367,9852,83,34,77,-100,9
5,C.J. Beathard,Iowa,Big Ten,13,10,1-9-0,225,393,2682,12,...,205,4,454,782,5562,40,19,224,429,10
6,Joshua Dobbs,Tennessee,SEC,4,0,0-0-0,2,3,19,0,...,-4,0,614,999,7138,53,29,438,2160,32
7,Nathan Peterman,Pittsburgh,ACC,8,4,1-3-0,68,130,548,3,...,73,1,398,662,5236,47,17,172,525,5
8,Brad Kaaya,Miami (FL),ACC,0,0,0-0-0,0,0,0,0,...,0,0,720,1188,9968,69,24,94,-386,4
9,Chad Kelly,Mississippi,SEC,1,0,0-0-0,0,0,0,0,...,-1,0,513,803,6858,50,21,203,958,16


In [11]:
df.to_csv('data/scraped_data.csv')